# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [1]:
import os
from FACET.Facet import Facet
os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

eegDataSet = "NiazyFMRI.edf"
#eegDataSet = "C:\\Users\\janik\\Downloads\\FMRIB_Data.set\\eegNiazy.edf"

f = Facet()


## Second Step import EEG Data

In [2]:
f.import_EEG(eegDataSet, rel_trig_pos=-0.01, bads=['EMG', 'ECG'])
f.find_triggers(r'\b1\b')
eeg = f.get_EEG()
#print triggers
print(eeg["triggers"])
event_id={'trigger':1}
f.export_as_bids(event_id)
f.import_from_bids(rel_trig_pos=-0.01, bads=['EMG', 'ECG'])
eeg = f.get_EEG()
#print channels
print(eeg["raw"].ch_names)

f.plot_EEG(title="after import")

Importing EEG with:
Channels ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2', 'AF4', 'AF3', 'FC2', 'FC1', 'CP1', 'CP2', 'PO3', 'PO4', 'FC6', 'FC5', 'CP5', 'EMG', 'ECG', 'Status']
Time Start: 0.0s
Time End: 162.99951171875s
Number of Samples: 333824
Sampling Frequency: 2048.0Hz
NiazyFMRI.edf
Stim-Kanäle gefunden: ['Status']
Channels after find trigger: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2', 'AF4', 'AF3', 'FC2', 'FC1', 'CP1', 'CP2', 'PO3', 'PO4', 'FC6', 'FC5', 'CP5', 'EMG', 'ECG', 'Status']
finding triggers
[58886, 59178, 59470, 59762, 60054, 60346, 60638, 60931, 61223, 61515, 61807, 62099, 62391, 62683, 62975, 63268, 63560, 63852, 64144, 64436, 64728, 65030, 65323, 65615, 65907, 66199, 66491, 66783, 67075, 67367, 67659, 67952, 68244, 68536, 68828, 69120, 69412, 69704, 69996, 70288, 70581, 70873, 71175, 71467, 71759, 72051, 72344, 72636, 72928,

## Third Step Preprocessing

In [ ]:
f.pre_processing()


## Fourth Step find triggers

In [ ]:

f.find_triggers(r'\btrigger\b', idx=0) # Using Niazys data
eeg = f.get_EEG()
#print triggers
print(eeg["triggers"])
#f.cut()

#f.find_triggers(r'.*TR.*') # Using Fischmeisters data

## Fourth step apply AAS

In [ ]:
f.apply_MNE_AAS(method="numpy")

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )
eeg = f.get_EEG()
#print triggers
print(eeg["triggers"])



## Sixth step Post-Processing

In [ ]:
f.downsample()
f.plot_EEG( title="after downsampling")
f.lowpass(h_freq=40)

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG(title="after lowpass")
f.export_EEG("NiazyFMRI_pyFacet_cleaned.edf")

## Eighth Evaluate Results

In [ ]:
f.find_triggers(r'\btrigger\b') # Using Niazys data
eeg_python = f.get_eeg()
f.add_to_evaluate(eeg_python, name="MNE_matrix") # Matrix will result in >1.0 RMS2 because of python mne inaccuracies


#matlab version
eeg_matlab = f.import_EEG('NiazyFMRI_matlab_cleaned.edf',rel_trig_pos=-0.01, bads=['EMG', 'ECG']) # EDF File may not save events # maybe use bids instead
eeg_matlab["raw_orig"]=eeg_python["raw_orig"]
eeg_matlab["tmin"]=eeg_python["tmin"]
eeg_matlab["tmax"]=eeg_python["tmax"]
eeg_matlab["triggers"]=eeg_python["triggers"]
f.lowpass(h_freq=40)
f.add_to_evaluate(f.get_eeg(), start_time=eeg_python["time_triggers_start"], end_time=eeg_python["time_triggers_end"], name="Matlab")

#normal version
f.import_EEG(eegDataSet, rel_trig_pos=-0.01, bads=['EMG', 'ECG'])
f.pre_processing()
f.find_triggers(r'\b1\b')
f.apply_MNE_AAS(method="normal")
f.remove_artifacts()
f.downsample()
f.lowpass(h_freq=40)
f.find_triggers(r'\b1\b')
f.add_to_evaluate(f.get_eeg(), name="Normal")
f.export_EEG("NiazyFMRI_pyFacet_cleaned_new.edf")



results = f.evaluate(plot=True,measures=["SNR", "RMS", "RMS2", "MEDIAN"])
print(results)

f.export_as_bids(event_id=event_id)

